In [1]:
#!pip install google-cloud-speech

  Created wheel for googleapis-common-protos: filename=googleapis_common_protos-1.51.0-cp37-none-any.whl size=77603 sha256=ddfbd7520d71d4482c3bded072f2c587336348a4cad6c72d45b6a201e0052e6e
  Stored in directory: C:\Users\Administrator\AppData\Local\pip\Cache\wheels\2c\f9\7f\6eb87e636072bf467e25348bbeb96849333e6a080dca78f706
Successfully built googleapis-common-protos


In [2]:
#!pip install google-cloud-storage

In [3]:
#!pip install librosa

  Created wheel for librosa: filename=librosa-0.7.2-cp37-none-any.whl size=1612889 sha256=e0a6159489e779d2f0eb1c2aec44218ff3dc2e298a73e44e70b147873e9b1517
  Stored in directory: C:\Users\Administrator\AppData\Local\pip\Cache\wheels\4c\6e\d7\bb93911540d2d1e44d690a1561871e5b6af82b69e80938abef
  Created wheel for audioread: filename=audioread-2.1.8-cp37-none-any.whl size=23097 sha256=bb8c8c9252432bb0a8f55f2f736b7d2268238001703d6e059be002e6fbb1756d
  Stored in directory: C:\Users\Administrator\AppData\Local\pip\Cache\wheels\b9\64\09\0b6417df9d8ba8bc61a7d2553c5cebd714ec169644c88fc012
  Created wheel for resampy: filename=resampy-0.2.2-cp37-none-any.whl size=320724 sha256=0d5500b60716d4ee1edf20ef112725ebd73d03824728dd35f65b9ce1c685581e
  Stored in directory: C:\Users\Administrator\AppData\Local\pip\Cache\wheels\fa\c1\56\e0e12c6f7f3d2cdea9712b35136a2d40a7817c6210ec096485
Successfully built librosa audioread resampy


In [4]:
#!pip install textrank

  Created wheel for textrank: filename=textrank-0.1.0-cp37-none-any.whl size=3425 sha256=37d941de79c1a94c22d8333eec5059b057648e71e8bb3c86c63265a386a8c02e
  Stored in directory: C:\Users\Administrator\AppData\Local\pip\Cache\wheels\64\25\bf\e1b2f46da8146fe46171099ac23f084c17b97c3088cf3f7065
Successfully built textrank


In [1]:
#!pip install konlpy

In [ ]:
from flask import Flask, redirect, request, url_for
from werkzeug import secure_filename

import io
import os

from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

from google.cloud import speech_v1
from google.cloud.speech_v1 import enums
from google.cloud.speech_v1 import types
from google.cloud import storage

from textrank import KeysentenceSummarizer
from konlpy.tag import Komoran
from konlpy.tag import Okt

from wordcloud import WordCloud

import nltk
from nltk.corpus import stopwords
%matplotlib inline
import matplotlib
from IPython.display import set_matplotlib_formats
matplotlib.rc('font',family = 'Malgun Gothic')

set_matplotlib_formats('retina')

matplotlib.rc('axes',unicode_minus = False)

import librosa
import librosa.display
import IPython.display

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import pandas as pd

import soundfile as sf
import json
import requests
from nltk import Text

#GOOGLE_APPLICATION_CREDENTIALS=""

app = Flask(__name__)


@app.route('/fileupload', methods = ['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        
        # 파일업로드 jsp에 녹음파일 요청
        f = request.files['file']
        fname = str(secure_filename(f.filename))
        f.save('./audio/'+fname)
        
        # 오디오 first_cut파일 담을 dataframe생성
        columns = ['cut_audio']
        cut_df = pd.DataFrame(columns=columns)

        # 오디오 파일 경로에서 파일 이름만 빼내기
        audio_path = './audio/'+fname
        (file_dir, file_id) = os.path.split(audio_path)

        y, sr = librosa.load(audio_path, sr=16000)
        X = 60 # 끊을 초
        t = len(y)//sr//X+1

        for i in range(t):
            # 오디오 자르기1
            cut_y = y[round(i*X*16000) : round((i+1)*X*16000)]

            # 오디오 cut파일 저장 경로1
            fname2 = file_id.split('.wav')[0]
            cut_audio = './cutfirst/{}{}.wav'.format(fname2, i)

            # cutfirst에 저장
            sf.write(cut_audio, cut_y, 16000, format='wav', subtype='PCM_16')

            # cut_df에 넣기
            cut_df.loc[i,['cut_audio']] = cut_audio

            i+=1

        import requests
        
        # 화자구분 API
        def vfilter(cut_audio):
            files = {
                'apiId': (None, 'dhguswl12303061dc811a'),
                'apiKey': (None, '0ad29b46083a42ed8700d23f2d90e78a'),
                'reqVoice': (cut_audio, open(cut_audio, 'rb')),
                }
            response = requests.post('https://api.maum.ai/api/dap/diarize/', files=files)
            return response


        #  오디오 second_cut파일 담을 dataframe생성
        columns = ['cut2_audio']
        cut2_df = pd.DataFrame(columns=columns)

        z = 0
        for i in range(len(cut_df)):
            cut_audio = cut_df['cut_audio'][i]

            # API 요청
            response = vfilter(cut_audio)

            # json
            dict = json.loads(response.text)

            y, sr = librosa.load(cut_audio, sr=16000)

            j = 0
            for d in dict['data']:
                if (d['endTime'] - d['startTime']) > 6:
                    # 오디오 자르기2
                    cut2_y = y[round(d['startTime']*16000) : round((d['endTime']+0.5)*16000)]

                    # 오디오 cut파일 저장 경로2
                    fname3 = cut_audio.split('.wav')[0].split('./cutfirst/')[1]
                    audio_path2 = './cutsecond/{}_{}.wav'.format(fname3, j)

                    # cutsecond에 저장
                    sf.write(audio_path2, cut2_y, 16000, format='wav', subtype='PCM_16')

                    # cut2_df에 넣기
                    cut2_df.loc[z,['cut2_audio']] = audio_path2

                    j+=1
                    z+=1

        # speech-to-text API
        def transcribe_audio(cut2_audio):
            client = speech.SpeechClient()

            with io.open(cut2_audio, 'rb') as f:
                content = f.read()
            audio = {"content": content}

            config = types.RecognitionConfig(
                encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
                language_code='ko-KR',
                audio_channel_count=1)

            response = client.recognize(config, audio)

            return response

        for i in range(len(cut2_df)):
            cut2_audio = cut2_df['cut2_audio'][i]

            # API 요청
            response = transcribe_audio(cut2_audio)

            for result in response.results:
                alternative = result.alternatives[0]

            if i%2 == 0:
                text = 'A : '+ alternative.transcript + '<br><br>'
                scriptNm = 'result'
                f = open('textResult/{}.txt'.format(scriptNm),'a')
                f.write(text)
                f.close()
            elif i%2 == 1:
                text = 'B : '+alternative.transcript + '<br><br>'
                scriptNm = 'result'
                f = open('textResult/{}.txt'.format(scriptNm),'a')
                f.write(text)
                f.close()


        # 오디오 경로
        audio_path = './audio/'+fname
        (file_dir, file_id) = os.path.split(audio_path)

        # 오디오 파일 구글스토리지에 저장
        def upload_blob(bucket_name, source_file_name, destination_blob_name):

            from google.cloud import storage

            """Uploads a file to the bucket."""
            storage_client = storage.Client()
            bucket = storage_client.get_bucket(bucket_name)
            blob = bucket.blob(destination_blob_name)

            blob.upload_from_filename(source_file_name)

        # 스토리지 저장 함수 실행
        upload_blob('test_p1', audio_path, file_id)

        # speech-to-text
        def transcribe_gcs(gcs_uri):

            client = speech.SpeechClient()
            audio = types.RecognitionAudio(uri=gcs_uri)
            config = types.RecognitionConfig(
                encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
                sample_rate_hertz=16000, audio_channel_count=2,
                language_code='ko-KR',
                enable_automatic_punctuation=True)

            operation = client.long_running_recognize(config, audio)
            response = operation.result()

            return response
        
        # speech-to-text 함수 실행
        response = transcribe_gcs("gs://test_p1/"+file_id)

        text2 = ''
        for result in response.results:
            text2 += result.alternatives[0].transcript


        texts = text2.split('. ')

        #texts



        # komoran_tokenizer : 형태소분석기 선언, 정의
        komoran = Komoran()
        def komoran_tokenizer(sent):
            words = komoran.pos(sent, join=True)
            words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
            return words

        # komoran_tokenizer : 형태소분석기 사용
        summarizer = KeysentenceSummarizer(
            tokenize = komoran_tokenizer,
            min_sim = 0.3,
            verbose = False
        )

        # title(안건) 뽑는 함수

        title = ''
        bias = np.ones(len(texts))
        bias[1:3] = 20

        keysents = summarizer.summarize(texts, topk=2, bias=bias)
        for _, _, sent in keysents:
            print(sent)
            title += (sent + '&#10;')
            texts.remove(sent) 

        # title 저장(titleNM.txt)
        titletext = title #안건 내용
        titleNm = 'title' #안건 파일이름
        t = open('textResult/{}.txt'.format(titleNm),'w') #안건파일 저장
        t.write(titletext)
        t.close()

        # final(결론) 뽑기
        final = ''
        bias2 = np.ones(len(texts))
        bias2[-1] = 20        

        keysents = summarizer.summarize(texts, topk=1, bias=bias2)
        for _, _, sent in keysents:
            print(sent)
            final += (sent + '&#10;')
            texts.remove(sent)        

        # final저장(finalNm.txt)
        finaltext = final #결론 내용
        finalNm = 'final' #결론 파일이름
        f = open('textResult/{}.txt'.format(finalNm),'w') #결론파일 저장
        f.write(finaltext)
        f.close()

        #전체 Script 요약하기(핵심문장 뽑기)
        summary = ''
        keysents = summarizer.summarize(texts, topk=15)
        for _, _, sent in keysents:
            summary += (sent + '&#10;')

        # summary 저장 (summarNm.txt)
        summartext = summary #핵심요약문장 내용
        summarNm = 'summary' #핵심요약문장 파일이름
        s = open('textResult/{}.txt'.format(summarNm),'w') #핵심요약문장 파일 저장
        s.write(summartext)
        s.close()

        # 워드클라우드에 불필요한 단어 없애기
        # 형태소 분리
        okt = Okt()

        #불용어 처리
        stopwords = pd.read_csv("./stopwords/stopwords.txt")

        stopwords = stopwords['불용어'].tolist()

        okt_morphs = okt.pos(text2)
        Noun_words = okt.nouns(text2)

        for word, pos in okt_morphs:
            if pos == 'Noun':
                Noun_words.append(word)

        unique_Noun_words = set(Noun_words)
        for word in unique_Noun_words:
            if word in stopwords:
                while word in Noun_words: Noun_words.remove(word)

        # 워드 클라우드로 표현
        FONT_PATH = 'c:/Windows/Fonts/nanum/NanumBarunGothic.ttf'
        noun_text = ''
        for word in Noun_words:
            noun_text = noun_text +' '+word

        fig = plt.figure()
        wordcloud = WordCloud(font_path=FONT_PATH, max_words=70, background_color = 'white',width = 480, height = 280).generate(noun_text) # generate() 는 하나의 string value를 입력 받음
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")


        # 워드클라우드 저장
        wcNm = 'wc'+fname2
        fig.savefig('C:/Users/vdi02/Desktop/ViewRojoct/Viewroject/WebContent/result/wordcloud{}.png'.format(wcNm))

        # scriptNm titleNm finalNm summarNm wcNm
        return redirect('http://localhost:8082/Viewroject/minutesWrite.jsp?path1='+scriptNm+'&path2='+titleNm+'&path3='+finalNm+'&path4='+summarNm+'&img='+wcNm)
        
if __name__ == "__main__":
    from werkzeug.serving import run_simple
    run_simple("localhost",9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)


오늘의 안건은 보건용마스크 및 손소독제 긴급 수급 조정조치 추진 계획입니다
국가 코로나 관련 대책 회의를 시작하도록 하겠습니다
내일 오늘의 결정사항은 정부합동 단속반 확대 긴급 수급 조정 조치 마스크 공급 확대입니다.


127.0.0.1 - - [27/Feb/2020 21:00:24] "POST /fileupload HTTP/1.1" 302 -
